In [7]:
from expla_graphs import *

/home/ubuntu/anaconda3/envs/baselineretriever/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
! pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.2 MB/s eta 0:00:00


In [8]:
conda activate baselineretriever


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd

In [10]:
PATH = '/home/ubuntu/G-Retriever/dataset/expla_graphs'
prompt = 'Question: Do argument 1 and argument 2 support or counter each other? Answer in one word in the form of \'support\' or \'counter\'.\n\nAnswer:'
all_texts = pd.read_csv(f'{PATH}/train_dev.tsv', sep='\t')

In [11]:
index=0
text = all_texts.iloc[index]
topk=3
question = f'Argument 1: {text.arg1}\nArgument 2: {text.arg2}\n{prompt}'
nodes = pd.read_csv(f'{PATH}/nodes/{index}.csv')
edges = pd.read_csv(f'{PATH}/edges/{index}.csv')


nodes=nodes['node_attr'].astype(str).tolist()
edges=edges['edge_attr'].astype(str).tolist()

In [26]:
def BM25(query, nodes,edges, topk):
  # Retrieve with BM25

    tokenizer = AutoTokenizer.from_pretrained('facebook/spar-wiki-bm25-lexmodel-query-encoder')
    query_encoder = AutoModel.from_pretrained('facebook/spar-wiki-bm25-lexmodel-query-encoder')
    context_encoder = AutoModel.from_pretrained('facebook/spar-wiki-bm25-lexmodel-context-encoder')

    query_input = tokenizer(query, padding=True, truncation=True, return_tensors='pt')
    nodes_input = tokenizer(nodes, padding=True, truncation=True, return_tensors='pt')
    edges_input = tokenizer(edges, padding=True, truncation=True, return_tensors='pt')

    # Compute embeddings: take the last-layer hidden state of the [CLS] token
    query_emb = query_encoder(**query_input).last_hidden_state[:, 0, :]
    nodes_emb = context_encoder(**nodes_input).last_hidden_state[:, 0, :]
    edges_emb = context_encoder(**edges_input).last_hidden_state[:, 0, :]

    nodes_scores = query_emb @ nodes_emb.T
    _, topk_nodes_indices = torch.topk(nodes_scores, topk, largest=True)
    print(topk_nodes_indices)

    edges_scores = query_emb @ edges_emb.T
    _, topk_edges_indices = torch.topk(edges_scores, topk, largest=True)

    selected_nodes = " ".join([nodes[idx] for idx in topk_nodes_indices[0]])
    selected_edges = " ".join([edges[idx] for idx in topk_edges_indices[0]])
    answer = selected_nodes + " " + selected_edges 

    return answer


In [40]:
prompts=BM25(question, nodes, edges, topk) 
with open(f'/home/ubuntu/G-Retriever/baseline/BM25/expla_graphs/top{topk}/{index}.txt', 'w', encoding='utf-8') as file:
    file.write(prompts)
print("文件写入完成，并已成功关闭。")

tensor([[4, 0, 1]])
文件写入完成，并已成功关闭。


In [31]:
def sentence_transformer_models(query, nodes,edges, topk,model):
    
    model = SentenceTransformer(model)

    query_emb = model.encode(query)
    nodes_emb = model.encode(nodes)
    edges_emb = model.encode(edges)
    nodes_scores = torch.tensor(query_emb @ nodes_emb.T)
    edges_scores = torch.tensor(query_emb @ edges_emb.T)
    _, topk_nodes_indices = torch.topk(nodes_scores, topk, largest=True)
    _, topk_edges_indices = torch.topk(edges_scores, topk, largest=True)

    selected_nodes = " ".join([nodes[idx] for idx in topk_nodes_indices])
    selected_edges = " ".join([edges[idx] for idx in topk_edges_indices])
    answer= selected_nodes + " "+ selected_edges 

    return answer

In [32]:
for model in ['sentence-transformers/all-MiniLM-L12-v2', 'sentence-transformers/LaBSE','nthakur/mcontriever-base-msmarco']:
    prompts=sentence_transformer_models(question, nodes, edges, topk,model) 
    print(prompts)

cannabis marijuana legal causes synonym of capable of
marijuana cannabis legal synonym of causes capable of
cannabis marijuana legal synonym of causes capable of


In [34]:
import os

In [35]:
os.makedirs(f'/home/ubuntu/G-Retriever/baseline/BM25/expla_graphs/top{topk}', exist_ok=True)

In [ ]:

    
    for topk in [3,5,10]:
        os.makedirs(f'baseline/BM25/expla_graphs/top{topk}', exist_ok=True) #'baseline/BM25/top3
        for model in ['sentence-transformers/all-MiniLM-L12-v2', 'sentence-transformers/LaBSE','nthakur/mcontriever-base-msmarco']:
            os.makedirs(f'baseline/{model}/expla_graphs/top{topk}', exist_ok=True)
        for index in len(all_texts):
            text = all_texts.iloc[index]
            question = f'Argument 1: {text.arg1}\nArgument 2: {text.arg2}\n{prompt}'
            nodes = pd.read_csv(f'{PATH}/nodes/{index}.csv')
            edges = pd.read_csv(f'{PATH}/edges/{index}.csv')

            prompts=BM25(question, nodes, edges, topk) 
            
            with open(f'baseline/BM25/top{topk}/{index}.txt', 'w', encoding='utf-8') as file:
                file.write(prompts)
            
            for model in ['sentence-transformers/all-MiniLM-L12-v2', 'sentence-transformers/LaBSE','nthakur/mcontriever-base-msmarco']:
                prompts=sentence_transformer_models(question, nodes,edges, topk,model)
                with open(f'baseline/{model}/top{topk}/{index}.txt', 'w', encoding='utf-8') as file:
                    file.write(prompts)

In [22]:
def E5Base(query,nodes,edges,topk):
  def average_pool(last_hidden_states: Tensor,
                  attention_mask: Tensor) -> Tensor:
      last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
      return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


  # Each input text should start with "query: " or "passage: ", even for non-English texts.
  # For tasks other than retrieval, you can simply use the "query: " prefix.
  nodes_texts = combined_list = [f"query: {query}" ] + [f"passage: {n}" for n in nodes]
  edges_texts = combined_list = [f"query: {query}" ] + [f"passage: {e}" for e in edges]
  tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
  model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

  # Tokenize the input texts
  nodes_dict = tokenizer(nodes_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
  edges_dict = tokenizer(edges_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

  nodes_outputs = model(**nodes_dict)
  edges_outputs = model(**edges_dict)
  nodes_embeddings = average_pool(nodes_outputs.last_hidden_state, nodes_dict['attention_mask'])
  edges_embeddings = average_pool(edges_outputs.last_hidden_state, edges_dict['attention_mask'])

  # normalize embeddings
  nodes_embeddings = F.normalize(nodes_embeddings, p=2, dim=1)
  edges_embeddings = F.normalize(edges_embeddings, p=2, dim=1)
  nodes_scores = (nodes_embeddings[:1] @ nodes_embeddings[1:].T) * 100
  edges_scores = (edges_embeddings[:1] @ edges_embeddings[1:].T) * 100
  
  _, topk_nodes_indices = torch.topk(nodes_scores, topk, largest=True)
  _, topk_edges_indices = torch.topk(edges_scores, topk, largest=True)

  answer = " ".join([nodes[idx] for idx in topk_nodes_indices[0]])+" "+" ".join([edges[idx] for idx in topk_edges_indices[0]])

  return answer

In [23]:
topk=3
os.makedirs(f'/home/ubuntu/G-Retriever/baseline/E5Base/expla_graphs/top{topk}', exist_ok=True) #'baseline/BM25/top3
for index in range(len(all_texts)):
    text = all_texts.iloc[index]
    question = f'Argument 1: {text.arg1}\nArgument 2: {text.arg2}\n{prompt}'
    nodes = pd.read_csv(f'{PATH}/nodes/{index}.csv')
    edges = pd.read_csv(f'{PATH}/edges/{index}.csv')
    nodes=nodes['node_attr'].astype(str).tolist()
    edges=edges['edge_attr'].astype(str).tolist()
    prompts=E5Base(question,nodes,edges,topk)
    with open(f'/home/ubuntu/G-Retriever/baseline/E5Base/expla_graphs/top{topk}/{index}.txt', 'w', encoding='utf-8') as file:
            file.write(prompts)

/home/ubuntu/anaconda3/envs/baselineretriever/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ubuntu/anaconda3/envs/baselineretriever/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ubuntu/anaconda3/envs/baselineretriever/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ubuntu/anaconda3/envs/baselineretriever/

In [ ]:
def E5Base(query,nodes,edges,topk,E5_tokenizer,E5_model):
    def average_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    
    
    nodes_texts = combined_list = [f"query: {query}" ] + [f"passage: {n}" for n in nodes]
    nodes_dict = E5_tokenizer(nodes_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    nodes_outputs =E5_model(**nodes_dict)
    nodes_embeddings = average_pool(nodes_outputs.last_hidden_state, nodes_dict['attention_mask'])
    nodes_embeddings = F.normalize(nodes_embeddings, p=2, dim=1)
    nodes_scores = (nodes_embeddings[:1] @ nodes_embeddings[1:].T) * 100
    _, topk_nodes_indices = torch.topk(nodes_scores, topk, largest=True)
    selected_nodes = " ".join([nodes[idx] for idx in topk_nodes_indices[0]])
    
    edges_texts = combined_list = [f"query: {query}" ] + [f"passage: {e}" for e in edges]
    edges_dict = E5_tokenizer(edges_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    edges_outputs = E5_model(**edges_dict)
    edges_embeddings = average_pool(edges_outputs.last_hidden_state, edges_dict['attention_mask'])
    edges_embeddings = F.normalize(edges_embeddings, p=2, dim=1)
    edges_scores = (edges_embeddings[:1] @ edges_embeddings[1:].T) * 100
    _, topk_edges_indices = torch.topk(edges_scores, topk, largest=True)
    selected_edges = " ".join([edges[idx] for idx in topk_edges_indices[0]])
    answer= selected_nodes + " "+ selected_edges 
    return answer